In [26]:
# Install packages

# !pip install pandas
# !pip install numpy
# !pip install matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textwrap import wrap
from enum import Enum
import os

In [29]:
data_dir = 'data/fall-25'
results_dir = 'results/fall-25/values'

In [4]:
# Read in data

sos_fall_values_survey_df = pd.read_csv(f'{data_dir}/sos-values_fall-2025.csv')

In [46]:
# Reports to generate
# [x] Overall scores on all the questions (mean, std) (horizontal violin plot?)
# [X] Responses by major
# [X] By major, scores on all the questions (mean, std)
# [X] Demographics of interest by major: parents' education,
# days on campus, commute length, live with people who raised you, childcare
# responsibilities, LGBTQ+, transgender, income
# [ ] Some sort of wordcloud or LLM summarization of free-responses

class SosQuestions(Enum):
  VC1 = 'VC1'
  VC2 = 'VC2'
  VC3 = 'VC3'
  VC4 = 'VC4'
  VS1 = 'VS1'
  VS2 = 'VS2'
  VS3 = 'VS3'
  VS4 = 'VS4'
  VS5 = 'VS5'
  VV1 = 'VV1'
  VV2 = 'VV2'
  VV3 = 'VV3'
  VV4 = 'VV4'
  VV5 = 'VV5'
  VV6 = 'VV6'

Q_ID_TO_SOS_LABEL = {
    'Q1_1': SosQuestions.VC1.value,
    'Q1_2': SosQuestions.VC2.value,
    'Q1_3': SosQuestions.VC3.value,
    'Q1_4': SosQuestions.VC4.value,
    'Q5_1': SosQuestions.VS1.value,
    'Q5_2': SosQuestions.VS2.value,
    'Q5_3': SosQuestions.VS3.value,
    'Q5_4': SosQuestions.VS4.value,
    'Q5_5': SosQuestions.VS5.value,
    'Q10_1': SosQuestions.VV1.value,
    'Q10_2': SosQuestions.VV2.value,
    'Q10_3': SosQuestions.VV3.value,
    'Q10_4': SosQuestions.VV4.value,
    'Q10_5': SosQuestions.VV5.value,
    'Q10_6': SosQuestions.VV6.value,
}

SOS_LABEL_TO_TEXT = {
    SosQuestions.VC1.value: '...helps you feel comfortable asking questions and making comments in class?',
    SosQuestions.VC2.value: '...believes in your ability to learn and  succeed?',
    SosQuestions.VC3.value: '...makes you feel like part of the classroom community?',
    SosQuestions.VC4.value: '...values you for what you bring to class through your own personal experiences, inside and outside of class?',
    SosQuestions.VS1.value: '...clearly explains what you are expected to learn in the class?',
    SosQuestions.VS2.value: '...clearly explains the work to be done on all assignments/activities?',
    SosQuestions.VS3.value: '...provides assignments/activities that are useful in helping you learn the course material?',
    SosQuestions.VS4.value: '...provides feedback that is helpful for your learning?',
    SosQuestions.VS5.value: '...makes the class challenging in a way that is just right?',
    SosQuestions.VV1.value: '...is excited about the subject and shares this excitement with the class?',
    SosQuestions.VV2.value: '...uses in-class problem solving, and other interactive approaches?',
    SosQuestions.VV3.value: '...creates an environment where you interact with and learn from other students?',
    SosQuestions.VV4.value: '...encourages students to think critically and question assumptions?',
    SosQuestions.VV5.value: '...relates the course material to real-world applications?',
    SosQuestions.VV6.value: '...encourages students to take time to reflect on their own learning?',
}

SOS_LABEL_CATEGORIES = {
    'CLIMATE': [
      SosQuestions.VC1.value,
      SosQuestions.VC2.value,
      SosQuestions.VC3.value,
      SosQuestions.VC4.value
    ],
    'STRUCTURE': [
      SosQuestions.VS1.value,
      SosQuestions.VS2.value,
      SosQuestions.VS3.value,
      SosQuestions.VS4.value,
      SosQuestions.VS5.value
    ],
    'VIBRANCY': [
      SosQuestions.VV1.value,
      SosQuestions.VV2.value,
      SosQuestions.VV3.value,
      SosQuestions.VV4.value,
      SosQuestions.VV5.value,
      SosQuestions.VV6.value
    ]
}

LIKERT_TEXT_TO_VAL = {
    'Extremely important': 5,
    'Very important': 4,
    'Moderately important': 3,
    'Slightly important': 2,
    'Not at all important': 1,
}

class DemographicLabels(Enum):
  UNDERGRAD_GRAD = 'UNDERGRAD-GRAD'
  YEARS_AT_CSULA = 'YEARS_AT_CSULA'
  PARENTS_EDUCATION = 'PARENTS_EDUCATION'
  CAMPUS_OFF_CAMPUS = 'CAMPUS_OFF_CAMPUS'
  DAYS_ON_CAMPUS = 'DAYS_ON_CAMPUS'
  EMPLOYMENT = 'EMPLOYMENT'
  CHILDCARE = 'CHILDCARE'
  COMMUTE_LENGTH = 'COMMUTE_LENGTH'
  MAJOR = 'MAJOR'
  RACE_ETHNICITY = 'RACE_ETHNICITY'
  HOUSEHOLD_INCOME = 'HOUSEHOLD_INCOME'
  DISABILITY = 'DISABILITY'
  LGBTQIA = 'LGBTQIA'
  TRANSGENDER = 'TRANSGENDER'
  GENDER_IDENTITY = 'GENDER_IDENTITY'

Q_ID_TO_DEMOGRAPHIC_LABELS = {
    'Q1': 'UNDERGRAD-GRAD',
    'Q3': 'YEARS_AT_CSULA',
    'Q4': 'PARENTS_EDUCATION',
    'Q5': 'CAMPUS_OFF_CAMPUS',
    'Q6': 'DAYS_ON_CAMPUS',
    'Q7': 'EMPLOYMENT',
    'Q2.1': 'CHILDCARE',
    'Q1.2': 'COMMUTE_LENGTH',
    'Q2.2': 'MAJOR',
    'Q1.5': 'RACE_ETHNICITY',
    'Q2.4': 'HOUSEHOLD_INCOME',
    'Q3.2': 'DISABILITY',
    'Q6.1': 'LGBTQIA',
    'Q7.1': 'TRANSGENDER',
    'Q8.2': 'GENDER_IDENTITY'
}

RACE_ETHNICITY_TO_CLEAN_VALUE = {
  'Hispanic / Latinx': 'Hispanic - Latinx',
  'Asian / Asian American': 'Asian - Asian American',
  'Prefer not to say': 'Prefer not to say',
  'African American / Black': 'African American - Black',
  'Middle Eastern / North African': 'Middle Eastern - North African',
  'Hispanic / Latinx,White / European': 'Hispanic - Latinx,White - European',
  'Hispanic / Latinx,Other': 'Hispanic - Latinx,Other',
  'White / European': 'White - European',
  'Hispanic / Latinx,Native American / American Indian / Indigenous American / Alaskan Native,White / European': 'Hispanic - Latinx,Native American - American Indian - Indigenous American - Alaskan Native,White - European',
  'Middle Eastern / North African,White / European': 'Middle Eastern - North African,White - European',
  'Other': 'Other',
  'Asian / Asian American,White / European': 'Asian - Asian American,White - European',
  'African American / Black,Asian / Asian American': 'African American - Black,Asian - Asian American',
  'Hispanic / Latinx,Native American / American Indian / Indigenous American / Alaskan Native': 'Hispanic - Latinx,Native American - American Indian - Indigenous American - Alaskan Native',
  'African American / Black,Hispanic / Latinx,White / European': 'African American - Black,Hispanic - Latinx,White - European',
  'Native American / American Indian / Indigenous American / Alaskan Native': 'Native American - American Indian - Indigenous American - Alaskan Native',
}

HOUSEHOLD_INCOME_ORDER = [
    'Prefer not to say',
    'Unsure',
    '< $25,000 / year',
    '$25,000 - $50,000 / year',
    '$50,000 - $75,000 / year',
    '$75,000 - $100,000 / year',
    '$100,000 - $125,000 / year',
    '$125,000 - $150,000 / year',
    '> $150,000 / year'
]

COMMUTE_LENGTH_ORDER = [
    'Less than 30 mins',
    'Between 30 mins and 1 hr',
    'Between 1 hr and 2 hrs',
    'More than 2 hrs'
]

EMPLOYMENT_ORDER = [
    'Not employed',
    'Employed part-time: less than 10 hours per week',
    'Employed part-time: 10-20 hours per week',
    'Employed part-time: 20-30 hours per week',
    'Employed part-time: 30-40 hours per week',
    'Employed full-time: 40+ hours per week',
]

PARENTS_EDUCATION_ORDER = [
    'Prefer not to say',
    'Unsure',
    'I do not have a parent with any college experience',
    'I do not have a parent who has received at least a 4-year college degree, but I have a parent who attended some college',
    'I have a parent who has received at least a 4-year college degree'
]

In [56]:
def clean_data(df, columns_to_select):
  df_valid_response = df[df['DistributionChannel'] == 'anonymous'].copy()
  df_clean = df_valid_response[2:]

  # Rename columns and reformat SOS responses
  df_clean.rename(columns=Q_ID_TO_SOS_LABEL, inplace=True)
  df_clean.rename(columns=Q_ID_TO_DEMOGRAPHIC_LABELS, inplace=True)
  df_clean.replace(LIKERT_TEXT_TO_VAL, inplace=True)
  df_clean.replace(RACE_ETHNICITY_TO_CLEAN_VALUE, inplace=True)

  df_selected_responses = df_clean[columns_to_select]

  rows_to_drop = []
  # Drop rows that have only NaN for responses (no response) and less than 80%
  # of items filled out
  nan_df = df_selected_responses.isnull()
  all_nan_df = nan_df.sum(axis=1)
  all_nan_rows = all_nan_df[all_nan_df == nan_df.shape[1]].index
  less_than_eighty_filled_rows = all_nan_df[all_nan_df >= nan_df.shape[1] * 0.2].index
  rows_to_drop += list(all_nan_rows)
  rows_to_drop += list(less_than_eighty_filled_rows)

  # Drop rows with response time < 60 seconds
  df_clean['Duration (in seconds)'] = pd.to_numeric(df_clean['Duration (in seconds)'])
  low_response_drop_rows = df_clean[df_clean['Duration (in seconds)'] < 60].index
  rows_to_drop += list(low_response_drop_rows)

  return df_clean.drop(rows_to_drop)

def demographics_columns(survey_df):
  columns_to_select = list(DemographicLabels)
  demo_df = survey_df.copy()[columns_to_select]
  return demo_df

# def demographics_by_major_dfs(demographics_all_majors_df):
#   major_col = DemographicLabels.MAJOR.value
#   majors = demographics_all_majors_df.dropna(subset = [major_col])[major_col].unique()
#   demo_df_by_major = { major: demographics_all_majors_df[demographics_all_majors_df[major_col] == major] for major in majors }
#   return demo_df_by_major

def sos_columns(df):
  '''
  Return a dataframe of only the SOS questions and values as numbers.

  :param survey_df: The entire dataframe of survey data from Qualtrics.
  '''
  columns_to_select = list(map(lambda e: e.value, SosQuestions))
  # df_sos_responses = clean_data(survey_df, columns_to_select)
  df_sos_responses = df[columns_to_select]
  return df_sos_responses

def disaggregate_by_column(df, column_name):
  values = df.dropna(subset = [column_name])[column_name].unique()
  return { value: df[df[column_name] == value] for value in values }

def scores_by_disaggregate_dfs(disaggregated_dfs):
  return { category: sos_columns(df) for category, df in disaggregated_dfs.items() }

def sos_scores(scores_df):
  '''
  Calculate the SOS scores per-item for a dataframe.

  :param scores_df: The scores dataframe of survey data from Qualtrics (just
  the SOS questions and their numeric responses).
  '''
  score_vec_by_q = { col: np.array(scores_df[col]) for col in scores_df.columns }
  num_nan_per_q = { q_id: np.count_nonzero(np.isnan(score_vec)) for q_id, score_vec in score_vec_by_q.items() }
  score_vec_by_q_no_nan = { q_id: score_vec[~np.isnan(score_vec)] for q_id, score_vec in score_vec_by_q.items() }
  score_results_by_q = { q_id: {
      'n': len(score_vec),
      'mean': score_vec.mean(),
      'std': score_vec.std()
  } for q_id, score_vec in score_vec_by_q_no_nan.items() }
  return score_results_by_q, num_nan_per_q

def pretty_print_scores(scores_data):
  print("SOS Values Scores:")
  for sos_id,data in scores_data[0].items():
    print(f'\t- {sos_id}: ({data["mean"]}, {data["std"]}) (n={data["n"]})')
  print(f"Nonanswers per question:")
  for sos_id,count in scores_data[1].items():
    print(f'\t- {sos_id}: {count}')

def scores_for_table(scores_data):
  for i in range(len(scores_data[0])):
    sos_id = list(scores_data[0].keys())[i]
    data = scores_data[0][sos_id]
    count = scores_data[1][sos_id]
    return f'{sos_id}\t{data["mean"].round(2)}\t{data["std"].round(2)}\t{data["n"]}\t{count}'

def print_scores_for_table(scores_data):
  print(scores_for_table(scores_data))

def write_scores_for_table(scores_data, out_dir, out_file):
  with open(f'{out_dir}/{out_file}.txt', 'w') as f:
    f.write(scores_for_table(scores_data))

def graph_counts_histogram(categories, values, xlabel, ylabel, title):
  fix,ax = plt.subplots(figsize=(6,4))
  p = ax.bar(categories, values)
  ax.bar_label(p, values, label_type='center', color='#fff')
  ax.set_xticklabels(categories, rotation=-45, ha='left', fontsize=10)
  ax.set_xlabel(xlabel)
  ax.set_ylabel(ylabel)
  ax.set_title(title)
  plt.show()

def graph_income(demographics_df):
  demo_df = demographics_df.copy()
  incomes = HOUSEHOLD_INCOME_ORDER
  values = [ len(demo_df[demo_df['HOUSEHOLD_INCOME'] == income]) for income in incomes ]
  graph_counts_histogram(
      incomes,
      values,
      'Income',
      'Number of students',
      'Student household income (all majors)'
  )

def graph_disability(demographics_df):
  demo_df = demographics_df.copy()
  disability_states = demo_df.dropna(subset = ['DISABILITY'])['DISABILITY'].unique()
  values = [ len(demo_df[demo_df['DISABILITY'] == state]) for state in disability_states ]
  graph_counts_histogram(
      disability_states,
      values,
      'Disability status',
      'Number of students',
      'Student disability status (all majors)'
  )

def graph_childcare(demographics_df):
  demo_df = demographics_df.copy()
  childcare_states = demo_df.dropna(subset = ['CHILDCARE'])['CHILDCARE'].unique()
  values = [ len(demo_df[demo_df['CHILDCARE'] == state]) for state in childcare_states ]
  graph_counts_histogram(
      childcare_states,
      values,
      'Childcare responsibilities',
      'Number of students',
      'Student childcare responsibilities (all majors)'
  )

def graph_lgbtqia(demographics_df):
  demo_df = demographics_df.copy()
  lgbtqia_states = demo_df.dropna(subset = ['LGBTQIA'])['LGBTQIA'].unique()
  values = [ len(demo_df[demo_df['LGBTQIA'] == state]) for state in lgbtqia_states ]
  graph_counts_histogram(
      lgbtqia_states,
      values,
      'LGBTQIA+ status',
      'Number of students',
      'Students who identify as LGBTQIA+ (all majors)'
  )

def graph_transgender(demographics_df):
  demo_df = demographics_df.copy()
  transgender_states = demo_df.dropna(subset = ['TRANSGENDER'])['TRANSGENDER'].unique()
  values = [ len(demo_df[demo_df['TRANSGENDER'] == state]) for state in transgender_states ]
  graph_counts_histogram(
      transgender_states,
      values,
      'Transgender status',
      'Number of students',
      'Students who are transgender (all majors)'
  )

def graph_parents_education(demographics_df):
  demo_df = demographics_df.copy()
  education_states = PARENTS_EDUCATION_ORDER
  values = [ len(demo_df[demo_df['PARENTS_EDUCATION'] == state]) for state in education_states ]
  education_labels = [ '\n'.join(wrap(state, 30)) for state in education_states ]
  graph_counts_histogram(
      education_labels,
      values,
      'Parents education level',
      'Number of students',
      'Students parents educational attainment (all majors)'
  )

def graph_commute(demographics_df):
  demo_df = demographics_df.copy()
  commute_lengths = COMMUTE_LENGTH_ORDER
  values = [ len(demo_df[demo_df['COMMUTE_LENGTH'] == length]) for length in commute_lengths ]
  graph_counts_histogram(
      commute_lengths,
      values,
      'Commute length',
      'Number of students',
      'Student commute time to campus (all majors)'
  )

def graph_employment(demographics_df):
  demo_df = demographics_df.copy()
  employment_states = EMPLOYMENT_ORDER
  values = [ len(demo_df[demo_df['EMPLOYMENT'] == state]) for state in employment_states ]
  employment_labels = [ '\n'.join(wrap(state, 30)) for state in employment_states ]
  graph_counts_histogram(
      employment_labels,
      values,
      'Employment',
      'Number of students',
      'Student employment status (all majors)'
  )

def graph_responses_per_major(scores_by_major_dfs):
  fix,ax = plt.subplots(figsize=(6,4))
  responses_per_major = {
      major: 0 for major in scores_by_major_dfs.keys()
  }
  threshold = 0.8
  for major, scores in scores_by_major_dfs.items():
    for row in scores.iterrows():
      vec = np.array(row[1].values)
      percent_answered = np.isnan(vec.sum()) / len(vec)
      if percent_answered <= 1 - threshold:
        responses_per_major[major] += 1

  majors = list(responses_per_major.keys())
  responses = [ responses_per_major[major] for major in majors ]

  graph_counts_histogram(
      majors,
      responses,
      'Major',
      'Number of responses',
      'Students who responded to\nat least 80% of questions by major'
  )

  # p = ax.bar(majors, responses)
  # ax.bar_label(p, responses, label_type='center', color='#fff')
  # ax.set_xticklabels(majors, rotation=-45, ha='left', fontsize=10)
  # ax.set_xlabel('Major')
  # ax.set_ylabel()
  # ax.set_title()

  # plt.show()

def graph_scores(scores_df, out_dir='', out_file=''):
  '''
  Create three violin plots (climate, structure, vibrancy) for the
  scores in a dataframe for all SOS questions.

  :param scores_df: A dataframe with columns as SOS questions and values
  as likert score numbers per respondent
  '''
  for i in range(len(SOS_LABEL_CATEGORIES)):
    fig, ax = plt.subplots(figsize=(6,4))
    ax.set_xticks(range(1,6))
    ax.set_xlim([1, 5])
    ax.set_xticklabels(range(1,6))
    category = list(SOS_LABEL_CATEGORIES.keys())[i]

    temp_subplot = SOS_LABEL_CATEGORIES[category].copy()
    temp_subplot.reverse()
    subplot_data = scores_df[temp_subplot]
    question_texts = [ '\n'.join(wrap(SOS_LABEL_TO_TEXT[label], 30)) for label in temp_subplot ]
    ax.set_yticks(range(len(temp_subplot)))
    ax.set_yticklabels(question_texts)
    ax.set_xlabel('Likert Score\n(1 = "Not at all important", 5 = "Extremely important")')
    ax.set_ylabel('Question')
    ax.set_title(f'Responses for {category.lower()}\n"In your classes, how important is it that your professor..."')

    dataset = [ np.array(subplot_data[col]) for col in subplot_data.columns ]
    dataset = [ vec[~np.isnan(vec)] for vec in dataset ]

    ax.violinplot(dataset, orientation='horizontal', positions=range(len(temp_subplot)))

    if not out_dir and out_file:
      plt.show()
    else:
      plt.savefig(f'{out_dir}/{out_file}_{category}.png', bbox_inches='tight')
      plt.close()

def report_scores_per_category(df, column_name):
  disaggregated_dfs = disaggregate_by_column(df, column_name)
  results_path = f'{results_dir}/{column_name}'
  if not os.path.exists(results_path):
    os.mkdir(results_path)
  for value, disagg_df in disaggregated_dfs.items():
    print(f'========= {column_name}={value} =========')
    report_scores_for_df(disagg_df, results_path, f'{value}')

def report_scores_for_df(df, out_dir=results_dir, out_name='df_score'):
  sos_df = sos_columns(df)
  score_data = sos_scores(sos_df)
  # pretty_print_scores(score_data)
  # print_scores_for_table(score_data)
  write_scores_for_table(score_data, out_dir, f'{out_name}')
  graph_scores(sos_df, out_dir, f'{out_name}')

In [ ]:

clean_df = clean_data(sos_fall_values_survey_df, list(map(lambda e: e.value, SosQuestions)))
# print(clean_df)
report_scores_for_df(clean_df, results_dir, 'all-results')
report_scores_per_category(clean_df, DemographicLabels.MAJOR.value)
report_scores_per_category(clean_df, DemographicLabels.RACE_ETHNICITY.value)
report_scores_per_category(clean_df, DemographicLabels.GENDER_IDENTITY.value)
# graph_responses_per_major(scores_by_major_dfs(survey_by_major_dfs(sos_fall_values_survey_df)))

# demo_all_majors = demographics_all_majors_df(sos_fall_values_survey_df)
# demo_by_major = demographics_by_major_dfs(demo_all_majors)
# demo_by_major
# graph_income_all_majors(demo_all_majors)
# graph_disability_all_majors(demo_all_majors)
# graph_commute_all_majors(demo_all_majors)
# graph_employment_all_majors(demo_all_majors)
# graph_childcare_all_majors(demo_all_majors)
# graph_parents_education_all_majors(demo_all_majors)
# graph_transgender(demo_all_majors)
# graph_lgbtqia(demo_all_majors)

# for major,demo_data in demo_by_major.items():
#   print(f'======= MAJOR: {major} ========')
#   graph_income(demo_data)
#   graph_disability(demo_data)
#   graph_commute(demo_data)
#   graph_employment(demo_data)
#   graph_childcare(demo_data)
#   graph_parents_education(demo_data)
#   graph_transgender(demo_data)
#   graph_lgbtqia(demo_data)

# report_scores_per_race_eth(clean_df)
# report_scores_per_gender_id()

C:\Users\Nick\AppData\Local\Temp\ipykernel_29880\2002989910.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.rename(columns=Q_ID_TO_SOS_LABEL, inplace=True)
C:\Users\Nick\AppData\Local\Temp\ipykernel_29880\2002989910.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.rename(columns=Q_ID_TO_DEMOGRAPHIC_LABELS, inplace=True)
C:\Users\Nick\AppData\Local\Temp\ipykernel_29880\2002989910.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behav

========= MAJOR=Electrical Engineering =========
========= MAJOR=Civil Engineering =========
========= MAJOR=Computer Science =========
========= MAJOR=Aviation Administration =========
========= MAJOR=Mechanical Engineering =========
========= MAJOR=Computer Information Systems =========
========= RACE_ETHNICITY=Hispanic - Latinx =========
========= RACE_ETHNICITY=Asian - Asian American =========
========= RACE_ETHNICITY=Prefer not to say =========
========= RACE_ETHNICITY=African American - Black =========
========= RACE_ETHNICITY=Middle Eastern - North African =========
========= RACE_ETHNICITY=Hispanic - Latinx,White - European =========
========= RACE_ETHNICITY=Hispanic - Latinx,Other =========
========= RACE_ETHNICITY=White - European =========
========= RACE_ETHNICITY=Hispanic - Latinx,Native American - American Indian - Indigenous American - Alaskan Native,White - European =========
========= RACE_ETHNICITY=Middle Eastern - North African,White - European =========
========= RAC

In [43]:
ethnicity_combos = clean_df[DemographicLabels.RACE_ETHNICITY.value].unique()
for combo in ethnicity_combos:
  if not type(combo) is str and np.isnan(combo): continue
  print(f"'{combo}': '{combo.replace('/','-')}',")

'Hispanic / Latinx': 'Hispanic - Latinx',
'Asian / Asian American': 'Asian - Asian American',
'Prefer not to say': 'Prefer not to say',
'African American / Black': 'African American - Black',
'Middle Eastern / North African': 'Middle Eastern - North African',
'Hispanic / Latinx,White / European': 'Hispanic - Latinx,White - European',
'Hispanic / Latinx,Other': 'Hispanic - Latinx,Other',
'White / European': 'White - European',
'Hispanic / Latinx,Native American / American Indian / Indigenous American / Alaskan Native,White / European': 'Hispanic - Latinx,Native American - American Indian - Indigenous American - Alaskan Native,White - European',
'Middle Eastern / North African,White / European': 'Middle Eastern - North African,White - European',
'Other': 'Other',
'Asian / Asian American,White / European': 'Asian - Asian American,White - European',
'African American / Black,Asian / Asian American': 'African American - Black,Asian - Asian American',
'Hispanic / Latinx,Native American / Ame